In [1]:
from tumorsphere.core.simulation import Simulation

In [2]:
sim = Simulation(
    prob_stem=[0.7],
    prob_diff=[0],
    prob_supervivence_radiotherapy=[0.6],
    num_of_realizations=2,
    num_of_steps_per_realization=7,
)

In [4]:
sim.simulate_parallel(
    sql=False,
    output_dir="simple_radiotherapy",
)